In [1]:
import torch
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/detection/baseline/mmdetection/configs/object/trash.py')

# data root 설정
root='/opt/ml/detection/dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = '/opt/ml/stratified_kfold/cv_train_1.json' # train json 정보

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = '/opt/ml/stratified_kfold/cv_val_1.json' # train json 정보

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보

cfg.data.samples_per_gpu = 1

# seed 설정
cfg.seed = 2021

# gpu_ids 설정
cfg.gpu_ids = [0]

# work_dir 경로 설정
cfg.work_dir = './work_dirs/swinL_final'

# model num_classes 설정
cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
cfg.model.roi_head.bbox_head[2].num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=1, interval=1)




In [3]:
#print(cfg.data.train)
#torch.cuda.empty_cache()

In [4]:
cfg.pretrained = '/opt/ml/swin_large_patch4_window7_224_22k.pth'

In [5]:
datasets = [build_dataset(cfg.data.train)]
datasets[0]

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!



CocoDataset Train dataset with number of images 6295, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3022  | 1 [Paper]     | 4689  | 2 [Paper pack]  | 817   | 3 [Metal]   | 833   | 4 [Glass]    | 798   |
| 5 [Plastic]       | 2405  | 6 [Styrofoam] | 951   | 7 [Plastic bag] | 4565  | 8 [Battery] | 142   | 9 [Clothing] | 438   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [6]:
model = build_detector(cfg.model)
model.init_weights()

2022-04-06 05:38:10,875 - mmdet - INFO - load checkpoint from local path: /opt/ml/swin_large_patch4_window7_224_22k.pth
2022-04-06 05:38:11,568 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-04-06 05:38:11,594 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-04-06 05:38:11,603 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-04-06 05:38:11,710 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'

In [7]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2022-04-06 05:38:18,665 - mmdet - INFO - load checkpoint from local path: /opt/ml/detection/baseline/mmdetection/work_dirs/swinL/epoch_12.pth


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


2022-04-06 05:38:20,663 - mmdet - INFO - Start running, host: root@af755ca3f6ef, work_dir: /opt/ml/detection/baseline/mmdetection/work_dirs/swinL_final
2022-04-06 05:38:20,664 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
 -------------------- 
after_train_iter:
(ABOVE_NOR

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 3215/3215, 0.4 task/s, elapsed: 7588s, ETA:     0s

2022-04-06 09:03:14,821 - mmdet - INFO - Evaluating bbox...


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 3215/3215, 0.4 task/s, elapsed: 7576s, ETA:     0s

2022-04-06 22:43:02,246 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.25s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.29s).
Accumulating evaluation results...


2022-04-06 22:43:11,124 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.345
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.439
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.377
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.079
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.400
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.587
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.587
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.587
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.023
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.244
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.97s).
